# Assistants API - Knowledge Retrieval 

https://platform.openai.com/docs/assistants/tools/knowledge-retrieval

https://community.openai.com/t/new-assistants-api-a-potential-replacement-for-low-level-rag-style-content-generation/475677 

Watch:

https://youtu.be/5rcjGjgJNQc?t=600&si=d9OtX0nMi2Rv0fQV 


In [2]:
from openai import OpenAI
import json
from dotenv import load_dotenv, find_dotenv

_ : bool = load_dotenv(find_dotenv()) # read local .env file

In [3]:
client : OpenAI = OpenAI()

### Knowledge Retrieval

Retrieval augments the Assistant with knowledge from outside its model, such as proprietary product information or documents provided by your users. Once a file is uploaded and passed to the Assistant, OpenAI will automatically chunk your documents, index and store the embeddings, and implement vector search to retrieve relevant content to answer user queries.

https://platform.openai.com/docs/assistants/tools/knowledge-retrieval 



### How it works

The model then decides when to retrieve content based on the user Messages. The Assistants API automatically chooses between two retrieval techniques:

1. it either passes the file content in the prompt for short documents, or
2. performs a vector search for longer documents

Retrieval currently optimizes for quality by adding all relevant content to the context of model calls. We plan to introduce other retrieval strategies to enable developers to choose a different tradeoff between retrieval quality and model usage cost.

https://platform.openai.com/docs/assistants/tools/how-it-works


### Step 1: Upload the file and Create an Assistant

In [6]:
from openai.types.beta import Assistant

# Upload a file with an "assistants" purpose
file = client.files.create(
  file=open("zia_profile.pdf", "rb"),
  purpose='assistants'
)

print(file)



FileObject(id='file-2p7oFAVPX1IPQzOW35mP3SLI', bytes=48802, created_at=1700047815, filename='zia_profile.pdf', object='file', purpose='assistants', status='processed', status_details=None)


In [7]:
assistant: Assistant = client.beta.assistants.create(
  name="Student Support Assistant",
  instructions="You are a student support chatbot. Use your knowledge base to best respond to student queries about Zia U. Khan.",
  model="gpt-3.5-turbo-1106",
  tools=[{"type": "retrieval"}],
  file_ids=[file.id]
)


### Step 2: Create a Thread

In [8]:
from openai.types.beta.thread import Thread

thread: Thread  = client.beta.threads.create()

print(thread)


Thread(id='thread_UrU3fNWx1yMPGDMUoEuYDpbQ', created_at=1700048142, metadata={}, object='thread')


### Step 3: Add a Message to a Thread

In [9]:
from openai.types.beta.threads.thread_message import ThreadMessage

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="When and which city Zia U. Khan was born?"
)


### Step 4: Run the Assistant

In [10]:
from openai.types.beta.threads.run import Run

run: Run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Pakistani. The user is the student of PIAIC."
)


### Step 5: Check the Run status

In [16]:
run: Run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

print(run)


Run(id='run_2yATUyChhncfYC7AQh1MfGMt', assistant_id='asst_WZFG0ig0aVAGWMWfHrw3k5Ze', cancelled_at=None, completed_at=None, created_at=1700048421, expires_at=None, failed_at=1700048433, file_ids=['file-2p7oFAVPX1IPQzOW35mP3SLI'], instructions='Please address the user as Pakistani. The user is the student of PIAIC.', last_error=LastError(code='rate_limit_exceeded', message='Rate limit reached for gpt-3.5-turbo-1106 in organization org-9oP4S86HpNNbJMudY026ckbW on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.'), metadata={}, model='gpt-3.5-turbo-1106', object='thread.run', required_action=None, started_at=1700048421, status='failed', thread_id='thread_UrU3fNWx1yMPGDMUoEuYDpbQ', tools=[ToolAssistantToolsRetrieval(type='retrieval')])


### Step 6: Display the Assistant's Response

In [15]:
# from openai.resources.beta.threads.messages.messages import SyncCursorPage 

messages: list[ThreadMessage] = client.beta.threads.messages.list(
  thread_id=thread.id
)

for m in reversed(messages.data):
  print(m.role + ": " + m.content[0].text.value)


user: When and which city Zia U. Khan was born?
assistant: I'll need to open the file to find this information. Let me do that now.
